In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

In [2]:
import os
import shutil
import json

import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_transform as tft
from tensorflow_transform.saved import input_fn_maker, saved_transform_io
from tensorflow_transform.tf_metadata import metadata_io
import google.datalab.bigquery as dlbq

from configuration_2 import directories
from configuration_2 import ORDERED_TRAINING_COLUMNS, ORDERED_TRAINING_DEFAULTS, SIGNATURE_INT_COLUMNS, SIGNATURE_FLOAT_COLUMNS, SIGNATURE_METADATA

from signature_queries import sample_query

In [3]:
tf.logging.set_verbosity(tf.logging.INFO)

---

In [4]:
MODEL_DIR=os.path.join(os.environ['HOME'], "data", "model")
!ls $MODEL_DIR

In [5]:
# 
#  Uncomment if you want to start from scratch
#
!rm -rf $MODEL_DIR/*

# Training and Evaluation Data
Training and evaluation data should be provided in files already.

In [6]:
stage='sample'
gsdirs = directories('gs', stage)
localdirs = directories('local', stage)
gsdirs, localdirs

({'data': 'gs://going-tfx/sample/data',
  'metadata': 'gs://going-tfx/sample/metadata',
  'tmp': 'gs://going-tfx/sample/tmp'},
 {'data': '/tmp/atl_june/sample/data',
  'metadata': '/tmp/atl_june/sample/metadata',
  'tmp': '/tmp/atl_june/sample/tmp'})

#### Check if train and eval files exist
If not, please go back an run ```Processing_ATL_JUNE.ipynb```

In [7]:
localdatadir=localdirs['data']

In [8]:
!ls $localdatadir

atl_june_eval-00000-of-00003   atl_june_train-00011-of-00025
atl_june_eval-00001-of-00003   atl_june_train-00012-of-00025
atl_june_eval-00002-of-00003   atl_june_train-00013-of-00025
atl_june_train-00000-of-00025  atl_june_train-00014-of-00025
atl_june_train-00001-of-00025  atl_june_train-00015-of-00025
atl_june_train-00002-of-00025  atl_june_train-00016-of-00025
atl_june_train-00003-of-00025  atl_june_train-00017-of-00025
atl_june_train-00004-of-00025  atl_june_train-00018-of-00025
atl_june_train-00005-of-00025  atl_june_train-00019-of-00025
atl_june_train-00006-of-00025  atl_june_train-00020-of-00025
atl_june_train-00007-of-00025  atl_june_train-00021-of-00025
atl_june_train-00008-of-00025  atl_june_train-00022-of-00025
atl_june_train-00009-of-00025  atl_june_train-00023-of-00025
atl_june_train-00010-of-00025  atl_june_train-00024-of-00025


In [9]:
a_training_file = !ls $localdatadir/atl_june_train-00000-of-*
a_training_file = a_training_file[0]
!wc -l $a_training_file

1000 /tmp/atl_june/sample/data/atl_june_train-00000-of-00025


In [10]:
an_eval_file = !ls $localdatadir/atl_june_eval-00000-of-*
an_eval_file = an_eval_file[0]
!wc -l $an_eval_file

976 /tmp/atl_june/sample/data/atl_june_eval-00000-of-00003


---
#### Have a look into the training data file

This data is at the **training data** stage. It's got all and only the columns we want. Is has been normalized and integerized. We'll use ```tf.feature_column``` to further process categorical features.

In [11]:
probe = pd.read_csv(a_training_file, names=ORDERED_TRAINING_COLUMNS)

In [12]:
probe.sample(frac=1.0)[:2]

,ARR_DELAY,ARR_LAT,ARR_LON,DEP_DELAY,DEP_DOW,DEP_HOD,DEP_LAT,DEP_LON,DIFF_LAT,DIFF_LON,DISTANCE,MEAN_TEMP_ARR,MEAN_TEMP_DEP,MEAN_VIS_ARR,MEAN_VIS_DEP,WND_SPD_ARR,WND_SPD_DEP
826,132.0,41.33,-75.72,0.503632,4,20,33.63,-84.42,0.543593,0.882732,0.143711,0.610507,0.831933,0.204301,0.984375,0.005601,0.248062
824,3.0,30.47,-87.18,0.157385,4,10,33.63,-84.42,0.293766,0.759665,0.043596,0.581522,0.831933,0.489247,0.984375,0.004100,0.248062


In [13]:
probe.describe()

,ARR_DELAY,ARR_LAT,ARR_LON,DEP_DELAY,DEP_DOW,DEP_HOD,DEP_LAT,DEP_LON,DIFF_LAT,DIFF_LON,DISTANCE,MEAN_TEMP_ARR,MEAN_TEMP_DEP,MEAN_VIS_ARR,MEAN_VIS_DEP,WND_SPD_ARR,WND_SPD_DEP
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00,1000.00,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,12.653000,35.649160,-87.960920,0.164726,4.744000,14.378000,33.63,-84.42,0.412909,0.751279,0.137534,0.541755,0.639660,0.434478,0.727578,0.006493,0.349775
std,35.990776,5.288832,13.495345,0.082028,2.015571,4.654114,0.00,0.00,0.121666,0.144924,0.121488,0.132366,0.195845,0.074008,0.210591,0.002687,0.225363
min,-30.000000,18.330000,-157.920000,0.094431,1.000000,5.000000,33.63,-84.42,0.014493,0.000000,0.000000,0.106884,0.214286,0.086022,0.375000,0.000700,0.124031
25%,-8.000000,31.320000,-93.450000,0.123487,4.000000,10.000000,33.63,-84.42,0.313320,0.692333,0.062985,0.474638,0.483193,0.408602,0.515625,0.004700,0.209302
50%,1.000000,35.870000,-83.350000,0.128329,4.000000,14.000000,33.63,-84.42,0.417989,0.800795,0.106730,0.564312,0.684874,0.467742,0.671875,0.006201,0.248062
75%,18.000000,39.900000,-79.970000,0.169492,7.000000,18.000000,33.63,-84.42,0.510697,0.837092,0.150884,0.628623,0.816176,0.489247,0.984375,0.007901,0.604651
max,252.000000,61.170000,-64.970000,0.777240,7.000000,23.000000,33.63,-84.42,1.000000,0.998174,1.000000,0.947464,0.869748,0.489247,1.000000,0.024102,0.798450


---
# ```tf.data``` input function

#### This reads from a CSV of pre-processed data

In [14]:
def make_input_fn(filename, mode, batch_size):

    def _input_fn():
        dataset = tf.data.TextLineDataset(filename)

        def decode_csv(row):
            cols = tf.decode_csv(row, record_defaults=ORDERED_TRAINING_DEFAULTS)
            features = dict(zip(ORDERED_TRAINING_COLUMNS, cols))
            return features

        def pop_target(features):
            target = features.pop('ARR_DELAY')
            return features, target
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            dataset = dataset.shuffle(buffer_size=80000)
                
        dataset = (dataset.repeat()
                   .map(decode_csv)
                   .map(pop_target)
                   .batch(batch_size))

        return dataset.make_one_shot_iterator().get_next()
    
    return _input_fn

---
#### Verify the input_function's behaviour

In [15]:
with tf.Session() as sess:
    train_input_fn = make_input_fn(
        a_training_file, mode=tf.estimator.ModeKeys.TRAIN, batch_size=2)
    input = train_input_fn()
    _, res = sess.run([tf.global_variables_initializer(), input])
res

({'ARR_LAT': array([38.94, 26.68], dtype=float32),
  'ARR_LON': array([-77.46, -80.09], dtype=float32),
  'DEP_DELAY': array([0.12348669, 0.1283293 ], dtype=float32),
  'DEP_DOW': array([7, 7], dtype=int32),
  'DEP_HOD': array([21, 12], dtype=int32),
  'DEP_LAT': array([33.63, 33.63], dtype=float32),
  'DEP_LON': array([-84.42, -84.42], dtype=float32),
  'DIFF_LAT': array([0.4886128 , 0.20657925], dtype=float32),
  'DIFF_LON': array([0.8640464 , 0.83580333], dtype=float32),
  'DISTANCE': array([0.10286955, 0.10555956], dtype=float32),
  'MEAN_TEMP_ARR': array([0.58695644, 0.588768  ], dtype=float32),
  'MEAN_TEMP_DEP': array([0.41176477, 0.41176477], dtype=float32),
  'MEAN_VIS_ARR': array([0.4892473 , 0.37634405], dtype=float32),
  'MEAN_VIS_DEP': array([0.59375, 0.59375], dtype=float32),
  'WND_SPD_ARR': array([0.0070007 , 0.01350135], dtype=float32),
  'WND_SPD_DEP': array([0.79844964, 0.79844964], dtype=float32)},
 array([-3., -5.], dtype=float32))

---
# Feature engineering

#### Example: Bucketize latitude and longitude 
We can easily understand the range of values with the help of ```pandas.describe()```

In [16]:
query="""
select 
    distinct arrival_airport as airport, arrival_lat as lat, arrival_lon as lon 
from 
    `bigquery-samples.airline_ontime_data.flights`
"""
locations = dlbq.Query(query).execute().result().to_dataframe()
locations.describe()

,lat,lon
count,344.000000,344.000000
mean,38.491570,-98.531599
std,8.547964,21.746974
min,13.480000,-176.640000
25%,33.450000,-111.675000
50%,38.715000,-93.300000
75%,42.907500,-82.497500
max,71.280000,-64.800000


In [17]:
lat_boundaries = np.arange(10,80,5).tolist()
lat_boundaries

[10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75]

In [18]:
lon_boundaries = np.arange(-100, -55, 5).tolist()
lon_boundaries

[-100, -95, -90, -85, -80, -75, -70, -65, -60]

We're going to use those boundaries in the function below

---
#### Using tf feature_column api for bucketizing, crossing and embedding

In [19]:
def create_feature_columns():
    
    ################################################################
    #  Numerical columns for the pre-processed features
    ################################################################
    dep_delay = tf.feature_column.numeric_column('DEP_DELAY')
    mean_temp_dep = tf.feature_column.numeric_column('MEAN_TEMP_DEP')
    mean_vis_dep = tf.feature_column.numeric_column('MEAN_VIS_DEP')
    wnd_spd_dep = tf.feature_column.numeric_column('WND_SPD_DEP')
    mean_temp_arr = tf.feature_column.numeric_column('MEAN_TEMP_ARR')
    mean_vis_arr = tf.feature_column.numeric_column('MEAN_VIS_ARR')
    wnd_spd_arr = tf.feature_column.numeric_column('WND_SPD_ARR')
    diff_lat = tf.feature_column.numeric_column('DIFF_LAT')
    diff_lon = tf.feature_column.numeric_column('DIFF_LON')
    distance = tf.feature_column.numeric_column('DISTANCE')
    
    ################################################################
    #  Crossed and embedded
    ################################################################
    lat_boundaries = np.arange(10,80,5).tolist()
    lon_boundaries = np.arange(-100, -55, 5).tolist()
    
    cross_size = len(lat_boundaries) * len(lon_boundaries)

    arr_lat = tf.feature_column.numeric_column('ARR_LAT')
    arr_lat_b = tf.feature_column.bucketized_column(arr_lat, lat_boundaries)
    arr_lon = tf.feature_column.numeric_column('ARR_LON')
    arr_lon_b = tf.feature_column.bucketized_column(arr_lon, lon_boundaries)
    arr_geo_cross = tf.feature_column.crossed_column(['ARR_LAT', 'ARR_LON'], cross_size)
    arr_geo_emb = tf.feature_column.embedding_column(arr_geo_cross, 10)

    dep_lat = tf.feature_column.numeric_column("DEP_LAT")
    dep_lat_b = tf.feature_column.bucketized_column(dep_lat, lat_boundaries)
    dep_lon = tf.feature_column.numeric_column("DEP_LON")
    dep_lon_b = tf.feature_column.bucketized_column(dep_lon, lon_boundaries)
    dep_geo_cross = tf.feature_column.crossed_column(['DEP_LAT', 'DEP_LON'], cross_size)
    dep_geo_emb = tf.feature_column.embedding_column(dep_geo_cross, 10)

    dep_dow = tf.feature_column.categorical_column_with_identity("DEP_DOW", num_buckets=7)
    dep_hod = tf.feature_column.categorical_column_with_identity("DEP_HOD", num_buckets=24)
    dep_how = tf.feature_column.crossed_column(["DEP_HOD", "DEP_DOW"], 7*24)

    dep_how_emb = tf.feature_column.embedding_column(dep_how, 10)
    
    ################################################################
    #  Crossed and embedded
    ################################################################
    feature_columns = [
        dep_how_emb, arr_geo_emb, dep_geo_emb,
        dep_delay,
        mean_temp_dep, mean_temp_arr, mean_vis_dep, mean_vis_arr, wnd_spd_dep, wnd_spd_arr,
        diff_lat, diff_lon, distance
    ]
    
    return feature_columns

#### These feature columns will feed straight into the model

In [20]:
FEATURE_COLUMNS = create_feature_columns()

---
# Model function and custom estimator

#### The model function

In [21]:
def make_model_fn(feature_columns):
    
    def _model_fn(features, labels, mode, params):

        input_layer = tf.feature_column.input_layer( 
            features, feature_columns=feature_columns)

        #############################################################
        # This single line is the actual model
        #############################################################
        out = tf.layers.dense(input_layer, 1, activation=None)

        if mode == tf.estimator.ModeKeys.PREDICT:
            return tf.estimator.EstimatorSpec(mode, predictions=out)


        labels = tf.expand_dims(labels, -1)
        loss = tf.losses.mean_squared_error(labels, out)

        if mode == tf.estimator.ModeKeys.EVAL:    
            return tf.estimator.EstimatorSpec(
                mode=mode,
                loss = loss,
                #eval_metric_ops={'my_metric': }
            )

        else:
            optimizer = tf.train.GradientDescentOptimizer(params['learning_rate'])
            train_op = optimizer.minimize(loss, global_step=tf.train.get_or_create_global_step())

            return tf.estimator.EstimatorSpec(  
                mode,
                loss = loss,
                train_op = train_op)
        
    return _model_fn

#### The custom estimator

In [27]:
config = tf.estimator.RunConfig(model_dir=MODEL_DIR)

model_fn = make_model_fn(FEATURE_COLUMNS)

estimator = tf.estimator.Estimator(
        config=config,
        model_fn=model_fn,
        params={
            'learning_rate': 1e-3
        })

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5de4315e90>, '_model_dir': '/home/jupyter/data/model', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_global_id_in_cluster': 0, '_master': ''}


INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5de4315e90>, '_model_dir': '/home/jupyter/data/model', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_global_id_in_cluster': 0, '_master': ''}


---
#### The tft serving input function
The saved ```transform_fn``` of the preprocessing step is recovered and applied to the signature input, which is represented by placeholders.
This function will be exported together with the ```tf.feature_columns``` transforms after training, to support exactly the same preprocessing steps will can also be executed.
The estimator will first apply this function (actually, attach this graph) to the signature data coming at prediction time. 

In [71]:
def make_tft_serving_input_fn(metadata_dir):

    def _input_fn():
        # placeholders for all the raw inputs
        placeholders = {
            key: tf.placeholder(name = key, shape=[None], dtype=tf.int64)
            for key in SIGNATURE_INT_COLUMNS
        }
        placeholders.update({
            key: tf.placeholder(name = key, shape=[None], dtype=tf.float32)
            for key in SIGNATURE_FLOAT_COLUMNS
        })

        # transform using the saved model in transform_fn        
        transform_output = tft.TFTransformOutput(transform_output_dir=metadata_dir)
        features = transform_output.transform_raw_features(placeholders)
            
        return tf.estimator.export.ServingInputReceiver(features, placeholders)

    return _input_fn

---
# Training and Serving

#### At last, start the training!
First, we use the smaller sample data to verify the setup.

In [72]:
stage='sample'
gsdirs = directories('gs', stage)
localdirs = directories('local', stage)
localdatadir = localdirs['data']
METADATA_DIR=gsdirs['metadata']

If you want to start from scratch, execute this before the training:

In [73]:
shutil.rmtree(MODEL_DIR, ignore_errors=True)
!mkdir -p $MODEL_DIR
MODEL_DIR

'/home/jupyter/data/model'

In [74]:
METADATA_DIR=gsdirs['metadata']
exporter = tf.estimator.LatestExporter('exporter', make_tft_serving_input_fn(METADATA_DIR))

train_input_fn = make_input_fn(
    a_training_file, mode=tf.estimator.ModeKeys.TRAIN, batch_size=128)

eval_input_fn = make_input_fn(
    an_eval_file, mode=tf.estimator.ModeKeys.EVAL, batch_size=128)  

train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=1000)
eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn, steps = 50, exporters=exporter)

In [75]:
tf.estimator.train_and_evaluate(estimator, train_spec=train_spec, eval_spec=eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).


INFO:tensorflow:Running training and evaluation locally (non-distributed).


INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 1000 or save_checkpoints_secs None.


INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 1000 or save_checkpoints_secs None.


INFO:tensorflow:Calling model_fn.


...

INFO:tensorflow:SavedModel written to: /home/jupyter/data/model/export/exporter/temp-1541938383/saved_model.pb


INFO:tensorflow:SavedModel written to: /home/jupyter/data/model/export/exporter/temp-1541938383/saved_model.pb


INFO:tensorflow:Loss for final step: 711.58887.


INFO:tensorflow:Loss for final step: 711.58887.


({'global_step': 1000, 'loss': 1681.958},
 ['/home/jupyter/data/model/export/exporter/1541938383'])

---
#### Train with the full training set of ~300k records

In [76]:
stage='full'
gsdirs = directories('gs', stage)
localdirs = directories('local', stage)
localdatadir = localdirs['data']
METADATA_DIR=gsdirs['metadata']

In [77]:
gsdatadir = gsdirs['data']
res = !gsutil ls $gsdatadir
print("Displaying first 6 of {}.".format(gsdatadir))
res[:6]

Displaying first 6 of gs://going-tfx/full/data.


['gs://going-tfx/full/data/atl_june_eval-00000-of-00001',
 'gs://going-tfx/full/data/atl_june_train-00000-of-00005',
 'gs://going-tfx/full/data/atl_june_train-00001-of-00005',
 'gs://going-tfx/full/data/atl_june_train-00002-of-00005',
 'gs://going-tfx/full/data/atl_june_train-00003-of-00005',
 'gs://going-tfx/full/data/atl_june_train-00004-of-00005']

---
#### Copy the files to local storage

In [78]:
localdatadir = localdirs['data']
!mkdir -p $localdatadir
localdatadir

'/tmp/atl_june/full/data'

In [79]:
_ = !gsutil -m cp $gsdatadir/*eval* $localdatadir
_ = !gsutil -m cp $gsdatadir/*train* $localdatadir

In [80]:
all_train_files=!ls $localdatadir/atl_june_train-*-of-00005
all_eval_files=!ls $localdatadir/atl_june_eval-*-of-00001

In [81]:
all_train_files, all_eval_files

(['/tmp/atl_june/full/data/atl_june_train-00000-of-00005',
  '/tmp/atl_june/full/data/atl_june_train-00001-of-00005',
  '/tmp/atl_june/full/data/atl_june_train-00002-of-00005',
  '/tmp/atl_june/full/data/atl_june_train-00003-of-00005',
  '/tmp/atl_june/full/data/atl_june_train-00004-of-00005'],
 ['/tmp/atl_june/full/data/atl_june_eval-00000-of-00001'])

---
#### Create the estimator

In [82]:
def create_estimator(train_files, eval_files, model_fn, config, options):
    
    former_verbosity=tf.logging.get_verbosity()
    
    metadata_dir = options['metadata_dir']
    max_train_steps = options['max_train_steps']
    eval_steps = options['eval_steps']
    train_batch_size=options['train_batch_size']
    eval_batch_size=options['eval_batch_size']
   
    #################################################################################################
    #  Make the input functions from the file names passed herein
    #################################################################################################
    train_input_fn = make_input_fn(
        train_files, mode=tf.estimator.ModeKeys.TRAIN, batch_size=train_batch_size)

    eval_input_fn = make_input_fn(
        eval_files, mode=tf.estimator.ModeKeys.EVAL, batch_size=eval_batch_size)  

    #################################################################################################
    #  Make the serving input function and hand it to an exporter
    #################################################################################################
    exporter = tf.estimator.LatestExporter('exporter', make_tft_serving_input_fn(metadata_dir))

    #################################################################################################
    #  Create train and eval specification 
    #################################################################################################
    train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=max_train_steps)
    eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn, steps = eval_steps, exporters=exporter)    
    
    #################################################################################################
    #  And finally, the estimator
    #################################################################################################
    tf.logging.set_verbosity(tf.logging.WARN)
    estimator = tf.estimator.Estimator(
        config=config,
        model_fn=model_fn,
        params={
            'feature_columns': FEATURE_COLUMNS,
            'learning_rate': 1e-3
        })
    tf.logging.set_verbosity(former_verbosity)
    
    return estimator

In [83]:
config = tf.estimator.RunConfig(
    save_checkpoints_steps=1000,
    save_summary_steps=100,
    log_step_count_steps=500,
    model_dir=MODEL_DIR
)

options={
    'metadata_dir': METADATA_DIR,
    'max_train_steps': 10000,
    'eval_steps': 5,
    'train_batch_size': 128,
    'eval_batch_size': 1024
}
    
estimator = create_estimator(
    train_files=all_train_files,
    eval_files=all_eval_files, 
    model_fn=make_model_fn(feature_columns=FEATURE_COLUMNS),
    config = config,
    options=options)

In [84]:
!rm -rf $MODEL_DIR/*

In [42]:
#tf.logging.set_verbosity(tf.logging.WARN)
tf.estimator.train_and_evaluate(estimator, train_spec=train_spec, eval_spec=eval_spec)
#tf.logging.set_verbosity(tf.logging.INFO)
print("Finished.")

INFO:tensorflow:Running training and evaluation locally (non-distributed).


INFO:tensorflow:Running training and evaluation locally (non-distributed).


INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 1000 or save_checkpoints_secs None.


INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 1000 or save_checkpoints_secs None.


INFO:tensorflow:Calling model_fn.


...

INFO:tensorflow:SavedModel written to: /home/jupyter/data/model/export/exporter/temp-1541937613/saved_model.pb


INFO:tensorflow:SavedModel written to: /home/jupyter/data/model/export/exporter/temp-1541937613/saved_model.pb


INFO:tensorflow:Loss for final step: 1282.4895.


INFO:tensorflow:Loss for final step: 1282.4895.


Finished.


---
# Prediction 


#### Signature data for testing

First, let's get some test data. Now we need signature data, and that's what we have in Bigquery. Remember? 

In [79]:
SIGNATURE_COLUMNS = SIGNATURE_FLOAT_COLUMNS+SIGNATURE_INT_COLUMNS
signature_query=sample_query(SIGNATURE_COLUMNS, total=10000)
print(signature_query)
sample = dlbq.Query(signature_query).execute().result().to_dataframe()
print('Only {} examples. Showing first three:'.format(len(sample)))
sample[:3]


    SELECT
        DEP_LAT, DEP_LON, DEP_DELAY, MEAN_TEMP_DEP, MEAN_VIS_DEP, WND_SPD_DEP, ARR_LAT, ARR_LON, ARR_DELAY, MEAN_TEMP_ARR, MEAN_VIS_ARR, WND_SPD_ARR, DEP_DOW, DEP_T
    FROM 
        `going-tfx.examples.ATL_JUNE_SIGNATURE` 
    where
        MOD(ABS(FARM_FINGERPRINT(
            CONCAT(DATE,AIRLINE,ARR)
        )) + DEP_T, 10000) >= 0 
    and
        MOD(ABS(FARM_FINGERPRINT(
            CONCAT( DATE, AIRLINE, ARR)
        )) + DEP_T, 10000) < 1 
    
Only 40 examples. Showing first three:


,DEP_LAT,DEP_LON,DEP_DELAY,MEAN_TEMP_DEP,MEAN_VIS_DEP,WND_SPD_DEP,ARR_LAT,ARR_LON,ARR_DELAY,MEAN_TEMP_ARR,MEAN_VIS_ARR,WND_SPD_ARR,DEP_DOW,DEP_T
0,33.63,-84.42,-1.0,74.5,6.9,9.8,35.87,-78.78,7.0,78.6,9.0,4.6,1,1030
1,33.63,-84.42,-3.0,74.0,7.6,5.0,30.47,-87.18,-9.0,79.7,10.0,4.8,1,1025
2,33.63,-84.42,16.0,76.7,9.5,7.7,35.81,-83.99,14.0,74.2,9.9,9.8,1,1608


In [83]:
samplefile='/tmp/test.json'
a_record = sample.to_dict(orient='records')[0]
a_record.pop('ARR_DELAY')
with open(samplefile, 'w') as f:
    f.write(json.dumps(a_record))
!cat $samplefile

{"WND_SPD_DEP": 9.8, "DEP_DELAY": -1.0, "ARR_LAT": 35.87, "WND_SPD_ARR": 4.6, "MEAN_VIS_DEP": 6.9, "DEP_T": 1030.0, "MEAN_TEMP_ARR": 78.6, "DEP_LON": -84.42, "DEP_DOW": 1.0, "MEAN_VIS_ARR": 9.0, "MEAN_TEMP_DEP": 74.5, "DEP_LAT": 33.63, "ARR_LON": -78.78}

---
#### Local prediction with gcloud ml-engine

In [84]:
%%bash
latest=$(ls -tr /home/jupyter/data/model/export/exporter | tail -1)
echo Latest model: $latest
gcloud ml-engine local predict \
  --model-dir=/home/jupyter/data/model/export/exporter/$latest \
  --json-instances=/tmp/test.json

Latest model: 1541926921
OUTPUT
[14.867263793945312]


2018-11-11 09:12:35.836020: I tensorflow/core/common_runtime/process_util.cc:69] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.

